# Evaluation des résultats avec RAGAS

L'objectif de ce notebook est d'évaluer les résultats du système RAG créé précédemment.

## Installation des packages nécessaires

In [ ]:
!pip install -U -q langchain-community
!pip install --quiet langchain-google-genai
!pip install --quiet chromadb
!pip install -q pypdf
!pip install -q ragas
!pip install -q datasets

## Import des packages nécessaires

In [ ]:
import os
import getpass
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import PyPDFLoader
from datasets import Dataset

## Définition du modèle LLM et du modèle embedding

In [2]:
os.environ['GOOGLE_API_KEY'] = "XXXXXXXXXXXX" # Ici remplacez par votre API Key !!!
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.7, top_p=0.85)

## Définition du système RAG créé précédemment

In [3]:
def get_retriver(gemini_embeddings):

    vectorstore_disk = Chroma(
                         persist_directory="./chroma_db",
                         embedding_function=gemini_embeddings
                       )
    return vectorstore_disk.as_retriever(search_kwargs={"k": 3}) # k=3 signifie récupérer les 5 chunks les plus pertinents

def get_prompt():

    llm_prompt_template = """You are an assistant for question-answering tasks.
    Use the following context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use five sentences maximum and keep the answer concise.\n
    Question: {question} \nContext: {context} \nAnswer:"""

    return PromptTemplate.from_template(llm_prompt_template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def get_llm_response(question):
    retriever = get_retriver(gemini_embeddings)
    llm_prompt = get_prompt()
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | llm_prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain.invoke(question)

def load_file(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

def generate_vectorstore(file_path):

    pages = load_file(file_path)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunked_documents = text_splitter.split_documents(pages)

    print(f"Chargé {len(pages)} pages et créé {len(chunked_documents)} chunks.")
    vectorstore = Chroma.from_documents(
                        documents=chunked_documents,
                        embedding=gemini_embeddings,
                        persist_directory="./chroma_db"
                    )

    print(f"Base de données vectorielle créée/mise à jour dans ./chroma_db avec {len(chunked_documents)} chunks.")


## Chargement des documents sources

In [ ]:
generate_vectorstore(r"ici il faut remplacer par le path de votre fichier .pdf")

## Création de dataset de test

Pour évaluer la performance du système RAG, nous devons préparer un dataset de test. Ce dataset permettra au package ragas de calculer différentes métriques pour juger de la qualité de la récupération d'information et de la génération de réponse.

Ce dataset est structuré autour de plusieurs éléments :

- questions : La liste des questions posées au système RAG.

- ground_truths : Les réponses "idéales" ou attendues pour chaque question, basées sur les connaissances contenues dans les documents sources. C'est la vérité terrain pour la réponse.

- reference : Des extraits spécifiques des documents sources qui supportent les ground_truths. Ils représentent le contexte pertinent qui devrait idéalement être récupéré.

- answers : Les réponses réellement générées par notre système RAG pour chaque question.

- retrieved_contexts : Les morceaux de document (chunks) réellement récupérés par le retriever de notre système RAG pour chaque question.

En comparant les answers et retrieved_contexts produits par notre système avec les ground_truths et reference prédéfinis, ragas peut évaluer la fidélité de la réponse, la pertinence du contexte récupéré, etc.

In [ ]:
questions = [
    "define generative ai",
    "what is the date of rise of genai"
]

ground_truths = [
    ["Generative AI refers to artificial intelligence models capable of creating new content, such as text, images, music, or code, often based on patterns learned from large datasets."],
    ["The significant rise and public awareness of generative AI occurred recently, notably accelerating in late 2022 with the release of models like ChatGPT."]
]

reference = [
    "Generative AI is a category of artificial intelligence algorithms that generate new outputs based on the data they have been trained on. These models can produce diverse forms of content, including text, images, audio, and synthetic data.",
    "While generative AI concepts have existed for years, the field experienced a significant surge in capabilities and public attention starting in late 2022, largely driven by the release of highly capable large language models such as OpenAI's ChatGPT."
]

answers = []
contexts = []
retrieved_contexts = []

retriever = get_retriver(gemini_embeddings)
# Intégration des réponses du RAG aux questions de test
for query in questions:
  answers.append(get_llm_response(query))
  retrieved_contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

data = {
    "question": questions,
    "answer": answers,
    "reference": reference,
    "ground_truths": ground_truths,
    "retrieved_contexts": retrieved_contexts,
}
# Construction du dataset de test
dataset = Dataset.from_dict(data)

## Définition de la fonction de test :

Le package `RAGAS` permet de calculer différentes métriques pour juger de la qualité de la récupération d'information et de la génération de réponse :

- **Faithfulness (Fidélité) :** Cette métrique évalue la cohérence factuelle de la `answer` générée par rapport aux `retrieved_contexts`. Elle quantifie la proportion des affirmations contenues dans la réponse qui sont directement supportées par les informations extraites des documents sources. Un score élevé indique une faible tendance à l'hallucination, c'est-à-dire que le LLM ne génère pas d'informations non étayées par le contexte fourni.

- **Answer Relevancy (Pertinence de la Réponse) :** Cette métrique mesure le degré auquel la `answer` produite par le LLM adresse directement la question initiale de l'utilisateur. Elle évalue si la réponse est pertinente par rapport à l'intention de la question, indépendamment de la qualité du contexte récupéré.

- **Context Recall :** Cette métrique détermine la complétude des `retrieved_contexts`. Elle évalue dans quelle mesure les informations essentielles contenues dans les `ground_truths` sont présentes parmi les documents récupérés par le retriever. Un score élevé signifie que le retriever a réussi à extraire la majorité des informations pertinentes nécessaires pour formuler une réponse complète et correcte. Cette métrique requiert la disponibilité des `ground_truths` pour l'évaluation.

- **Context Precision (Précision du Contexte) :** Cette métrique évalue la pureté des `retrieved_contexts`. Elle mesure la proportion des documents récupérés qui sont effectivement pertinents pour répondre à la question. Un score élevé indique que le retriever minimise l'inclusion de `bruit` ou d'informations non pertinentes dans l'ensemble des documents passés au LLM.

- **Context Entity Recall :** Similaire au **Context Recall**, cette métrique se focalise spécifiquement sur la présence des entités clés identifiées dans les `ground_truths`. Elle permet d'évaluer si le retriever parvient à extraire les éléments nominatifs ou conceptuels centraux liés à la réponse attendue.

- **Answer Similarity (Similarité de la Réponse) :** Cette métrique compare la `answer` générée à la `ground_truths` en utilisant des techniques de similarité sémantique. Elle évalue si le sens global de la réponse générée est proche de celui de la `ground_truths`, même si la formulation textuelle diffère.

- **Answer Correctness (Exactitude de la Réponse) :** Cette métrique constitue une évaluation globale de l'exactitude factuelle de la `answer` générée par rapport aux `ground_truths`. C'est un indicateur clé de la performance globale du système RAG à produire des réponses véridiques. Cette métrique nécessite des ground_truths fiables pour une évaluation significative.

Cette liste de métriques n'est pas exhaustive ; vous pouvez consulter la [documentation officielle de Ragas](https://docs.ragas.io/en/stable/) pour la liste complète et choisir celles qui conviennent le mieux à l'évaluation de votre application spécifique.

In [6]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)
from datasets import Dataset
from langchain_google_genai import ChatGoogleGenerativeAI

def test_performance_rag(dataset):

  ragas_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
  gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

  result = evaluate(
      dataset,
      metrics=[
          faithfulness,
          context_precision,
          context_recall,
          answer_relevancy,
          context_entity_recall,
          answer_similarity,
          answer_correctness,
      ],
      llm=ragas_llm,
      embeddings=gemini_embeddings
  )

  return result.to_pandas()

Il est important de noter que la version gratuite de l'API Gemini impose une limite au nombre d'appels par minute (généralement entre 15 et 30 selon le modèle). Étant donné que Ragas effectue des appels intensifs au LLM pour calculer ses métriques, il est facile d'atteindre ce seuil. L'API Google gère automatiquement ce problème en mettant les appels en attente jusqu'à ce que de nouvelles requêtes soient disponibles. Par conséquent, l'exercice d'évaluation s'exécutera jusqu'au bout, mais il risque de prendre considérablement plus de temps, la majorité de ce temps étant consacré à l'attente de la disponibilité de nouveaux appels API.

In [ ]:
result = test_performance_rag(dataset)
#Afficher les résultats
result.head()